In [29]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, TextStreamer, QuantoConfig
import torch
from dotenv import load_dotenv
from datasets import load_dataset

data = load_dataset("squad", split="train")
load_dotenv("../.env")

base_model_path = "meta-llama/Llama-3.2-1B-Instruct"
tokenizer = AutoTokenizer.from_pretrained(base_model_path)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
quant_config = QuantoConfig(weights="int8")

def fetch_model(is_quant: bool=False):
    config = dict(
        pretrained_model_name_or_path=base_model_path,
        return_dict=True,
        low_cpu_mem_usage=True,
        # torch_dtype=torch.float16,
        device_map="auto",
        trust_remote_code=True,
        quantization_config=quant_config,
    )
    if is_quant:
        quant_cfg = {
            "quantization_config": quant_config,
        }
        config = config | quant_cfg

    model = AutoModelForCausalLM.from_pretrained(**config)
    if model.config.pad_token_id is None:
        model.config.pad_token_id = model.config.eos_token_id
    return model

def get_trainable_params(model) -> None:
    """
    Get the number of trainable parameters in the model.
    """
    params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"Total trainable parameters: {params}")

quant_model = fetch_model(is_quant=True)
base_model = fetch_model(is_quant=False)

In [30]:
base_model.config._name_or_path

'meta-llama/Llama-3.2-1B-Instruct'

In [50]:
pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokenizer,
    # torch_dtype=torch.float16,
    device_map="auto",
)

Device set to use mps


In [51]:
get_trainable_params(base_model)

Total trainable parameters: 262735872


In [52]:
get_trainable_params(quant_model)

Total trainable parameters: 262735872


In [54]:
from peft import (
    get_peft_model,
    prepare_model_for_kbit_training,
    LoraConfig
)

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    lora_dropout=0,
    target_modules=[
        "q_proj","k_proj","v_proj",
        "o_proj", "gate_proj", "up_proj",
        "down_proj"
    ],
    bias="none",
    task_type="CAUSAL_LM",
)

lora_base_model = prepare_model_for_kbit_training(base_model)
lora_base_model = get_peft_model(lora_base_model, lora_config)

In [55]:
get_trainable_params(lora_base_model)

Total trainable parameters: 11272192


In [56]:
get_trainable_params(quant_model)

Total trainable parameters: 262735872


In [58]:
11272192/262735872

0.04290313277054151

In [59]:
messages = [{"role": "user", "content": "Who is Vincent van Gogh?"}]

prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

outputs = pipe(prompt, max_new_tokens=120, do_sample=True)

print(outputs[0]["generated_text"])

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 07 May 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

Who is Vincent van Gogh?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Vincent Willem van Gogh (1853-1890) was a Dutch post-impressionist painter. He is widely regarded as one of the most influential and iconic artists of all time. Born in Groot-Zundert, Netherlands, Van Gogh was the eldest son of a Protestant pastor and grew up in a devout Christian household.

Van Gogh's early life was marked by struggles with mental illness, poverty, and hardship. He began his artistic training in the Netherlands, studying art in the Netherlands and later in Paris. He became known for his bold and expressive paintings, which often


In [60]:
from IPython.display import Markdown, display

messages = [
    {
        "role": "system",
        "content": "You are a skilled Python developer specializing in database management and optimization.",
    },
    {
        "role": "user",
        "content": "I'm experiencing a sorting issue in my database. Could you please provide Python code to help resolve this problem?",
    },
]

prompt = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

outputs = pipe(prompt, max_new_tokens=512, do_sample=True)

display(
    Markdown(
        outputs[0]["generated_text"].split(
            "<|start_header_id|>assistant<|end_header_id|>"
        )[1]
    )
)



I can help you with that. The problem you're experiencing is likely due to a bug in the sorting algorithm you're using. Here's a Python solution using the built-in `sorted()` function with a custom key function that handles your specific case:

### Problem: Sorting a List of Dictionaries

Let's say you have a list of dictionaries and you want to sort them based on a specific key.

### Solution

```python
def custom_sort_key(dictionary):
    """
    Custom key function to sort a dictionary based on a specific key.
    
    Args:
        dictionary (dict): The dictionary to be sorted.
    
    Returns:
        tuple: A tuple containing the index of the key and the value.
    """
    return (dictionary[key], dictionary[key].lower())

# Example usage:
data = [
    {"name": "John", "age": 25, "city": "New York"},
    {"name": "Alice", "age": 30, "city": "Los Angeles"},
    {"name": "Bob", "age": 20, "city": "Chicago"}
]

# Sort the list of dictionaries
sorted_data = sorted(data, key=custom_sort_key)

# Print the sorted list
for item in sorted_data:
    print(item)
```

### Explanation

In this code:

*   We define a custom key function `custom_sort_key` that takes a dictionary as an argument and returns a tuple containing the index of the key and the value.
*   We use the `sorted()` function with the `key` argument set to our custom key function. This tells Python to sort the list based on the values returned by the custom key function.
*   We apply this custom key function to each item in the list, which sorts the list based on the values of the specified key.

### Optimizations

If your list of dictionaries is very large, sorting it can be inefficient. In such cases, you can use the `sorted()` function with a `key` function that returns a lambda function that calculates the index of the key. This approach is faster than using a list comprehension with `sorted()`.

```python
def custom_sort_key(index, value):
    """
    Custom key function to sort a dictionary based on a specific key.
    
    Args:
        index (int): The index of the key.
        value: The value to be used for sorting.
    
    Returns:
        tuple: A tuple containing the index of the key and the value.
    """
    return (index,

In [64]:
data["train"][0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]}}

In [65]:
def apply_chat_template(example):
    answer = example['answers']['text']
    messages = [
        {"role": "user", "content": example['question']},
        {"role": "assistant", "content": answer}
    ]
    prompt = tokenizer.apply_chat_template(
        messages, tokenize=False, add_generation_prompt=True
    )
    return {"prompt": prompt}

templated_dataset = data.map(apply_chat_template)

In [25]:
from transformers import PreTrainedTokenizerFast


def apply_chat_template(
        input_dataset: dict, tokeniser: PreTrainedTokenizerFast
) -> dict:
    """Preprocess the input dataset to fit expected format.

    See: https://huggingface.co/docs/trl/en/sft_trainer for details.
    """
    answer = input_dataset['answers']['text']
    message_template = [
        {"role": "system", "content": input_dataset['context']},
        {"role": "user", "content": input_dataset['question'][0]},
        {"role": "assistant", "content": answer}
    ]
    # messages = tokeniser.apply_chat_template(
    #     message_template, tokenize=False, add_generation_prompt=True
    # )
    return {"messages": messages}

def apply_message_template(
    input_dataset: dict
) -> dict:
    """Preprocess the input dataset to fit expected format.

    See: https://huggingface.co/docs/trl/en/sft_trainer for details.
    """
    answer = input_dataset['answers']['text']
    if isinstance(input_dataset['answers']['text'], list):
        answer = input_dataset['answers']['text'][0]
    message_template = [
        {"role": "system", "content": input_dataset['context']},
        {"role": "user", "content": input_dataset['question']},
        {"role": "assistant", "content": answer}
    ]
    return {"messages": message_template}

templated_dataset = data.map(apply_message_template)

Map: 100%|██████████| 87599/87599 [00:02<00:00, 34745.72 examples/s]


In [26]:
templated_dataset[0]

{'id': '5733be284776f41900661182',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'To whom did the Virgin Mary allegedly appear in 1858 in Lourdes France?',
 'answers': {'text': ['Saint Bernadette Soubirous'], 'answer_start': [515]},
 'messages': [{'content': 'Architecturally, the sc

NameError: name 'base_model' is not defined

In [67]:
# tokenise the prompts
def tokenize_function(example):
    tokens = tokenizer(
        example['prompt'],
        padding="max_length",
        truncation=True,
        max_length=128
    )
    # Set padding token labels to -100 to ignore them in loss calculation
    tokens['labels'] = [
        -100 if token == tokenizer.pad_token_id else token for token in tokens['input_ids']
    ]
    return tokens

tokenised_dataset = templated_dataset.map(tokenize_function)
tokenised_dataset = tokenised_dataset.remove_columns(["title", "context", "question", "answers", "prompt"])

Map: 100%|██████████| 10570/10570 [00:02<00:00, 3587.33 examples/s]


In [68]:
# # https://www.mercity.ai/blog-post/guide-to-fine-tuning-llms-with-lora-and-qlora
# # https://huggingface.co/docs/diffusers/en/quantization/quanto
# for name, module in model.named_modules(): # https://www.datacamp.com/tutorial/fine-tuning-llama-3-2
#     print(f"name: {name}")
#     print(f"module: {module}")


In [31]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments
from datasets import load_dataset
from trl import SFTConfig, SFTTrainer
lora_base_model.train()
training_args = SFTConfig(
    output_dir = "cp",
    per_device_train_batch_size = 8,
    gradient_accumulation_steps = 8,
    per_device_eval_batch_size = 8,
    eval_accumulation_steps = 8,
    # optim = "paged_adamw_32bit",
    save_steps = 10,
    logging_steps = 10,
    learning_rate = 5e-5,
    max_grad_norm = 0.3,
    max_steps = 50,
    warmup_ratio = 0.03,
    eval_strategy="steps",
    lr_scheduler_type = "linear",
)
trainer = SFTTrainer(
    model=lora_base_model,
    args=training_args,
    train_dataset=tokenised_dataset["train"],
)

# Initialize Trainer
# trainer = Trainer(
#     model=lora_base_model,
#     args=training_args,
#     train_dataset=tokenised_dataset["train"],
#     eval_dataset=tokenised_dataset["validation"],
#     tokenizer=tokenizer)
#
# # Train the model
# trainer.train()
#
# # Save the model and tokenizer
# trainer.save_model("./fine-tuned-model")
# tokenizer.save_pretrained("./fine-tuned-model")

PyTorch: setting up devices
/var/folders/r8/jzt55db94034zypvblzs5zk80000gn/T/ipykernel_73868/1483521713.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
The following columns in the training set don't have a corresponding argument in `LlamaForCausalLM.forward` and have been ignored: id. If id are not expected by `LlamaForCausalLM.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 87,599
  Num Epochs = 2
  Instantaneous batch size per device = 20
  Total train batch size (w. parallel, distributed & accumulation) = 20
  Gradient Accumulation steps = 1
  Total optimization steps = 8,760
  Number of trainable parameters = 1,235,814,400


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [36]:
import pandas as pd
import json

path = "raw_data/ConvFinQA/train.json"
with open(path, "r") as file:
    data = json.load(file)

In [38]:
data[0]

{'pre_text': ['26 | 2009 annual report in fiscal 2008 , revenues in the credit union systems and services business segment increased 14% ( 14 % ) from fiscal 2007 .',
  'all revenue components within the segment experienced growth during fiscal 2008 .',
  'license revenue generated the largest dollar growth in revenue as episys ae , our flagship core processing system aimed at larger credit unions , experienced strong sales throughout the year .',
  'support and service revenue , which is the largest component of total revenues for the credit union segment , experienced 34 percent growth in eft support and 10 percent growth in in-house support .',
  'gross profit in this business segment increased $ 9344 in fiscal 2008 compared to fiscal 2007 , due primarily to the increase in license revenue , which carries the highest margins .',
  'liquidity and capital resources we have historically generated positive cash flow from operations and have generally used funds generated from operations

In [1]:
from peft import PeftModel
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
def load_lora_model(base_model_path, adapter_path):
    # First load the base model
    tokenizer = AutoTokenizer.from_pretrained(base_model_path)
    base_model = AutoModelForCausalLM.from_pretrained(
        base_model_path,
        device_map="auto",
        trust_remote_code=True
    )

    # Then load the LoRA adapter weights
    fine_tuned_model = PeftModel.from_pretrained(base_model, adapter_path)
    return fine_tuned_model, base_model, tokenizer
model_path = "../cp/checkpoint-60"  # Your output directory from training
base_model_path = "meta-llama/Llama-3.2-3B-Instruct"
ft_model, base_model, tokeniser = load_lora_model(base_model_path, model_path)

/Users/lloydhamilton/Documents/PersonalProjects.nosync/tomorro_llm_tech/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 2/2 [00:04<00:00,  2.11s/it]


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from datasets import load_dataset
val_data = load_dataset("squad")

ft_pipe = pipeline(
    "text-generation",
    model=ft_model,
    tokenizer=tokeniser,
    # torch_dtype=torch.float16,
    device_map="auto",
)

base_pipe = pipeline(
    "text-generation",
    model=base_model,
    tokenizer=tokeniser,
    # torch_dtype=torch.float16,
    device_map="auto",
)

Device set to use mps
The model 'PeftModelForCausalLM' is not supported for text-generation. Supported models are ['AriaTextForCausalLM', 'BambaForCausalLM', 'BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'LlamaForCausalLM', 'CodeGenForCausalLM', 'CohereForCausalLM', 'Cohere2ForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadModel', 'Data2VecTextForCausalLM', 'DbrxForCausalLM', 'DeepseekV3ForCausalLM', 'DiffLlamaForCausalLM', 'ElectraForCausalLM', 'Emu3ForCausalLM', 'ErnieForCausalLM', 'FalconForCausalLM', 'FalconMambaForCausalLM', 'FuyuForCausalLM', 'GemmaForCausalLM', 'Gemma2ForCausalLM', 'Gemma3ForConditionalGeneration', 'Gemma3ForCausalLM', 'GitForCausalLM', 'GlmForCausalLM', 'Glm4ForCausalLM', 'GotOcr2ForConditionalGeneration', 'GPT2LMHeadModel', 'GPT2LMHeadModel', 'GPTBigCodeForCausalLM', 'GPTNeoForCa

In [6]:
val_data["train"][1]

{'id': '5733be284776f4190066117f',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'What is in front of the Notre Dame Main Building?',
 'answers': {'text': ['a copper statue of Christ'], 'answer_start': [188]}}

In [4]:

messages = [{"role": "user", "content": "What is in front of the Notre Dame Main Building?"}]

prompt = tokeniser.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)
with torch.no_grad():
    outputs = base_pipe(prompt, max_new_tokens=120, do_sample=True)

print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 07 May 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

What is in front of the Notre Dame Main Building?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

The front of the Notre Dame Main Building is the courtyard, with the famous "Nightingale Bridge" in front.


In [5]:
with torch.no_grad():
    outputs = ft_pipe(prompt, max_new_tokens=120, do_sample=True)

print(outputs[0]["generated_text"])

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 07 May 2025

<|eot_id|><|start_header_id|>user<|end_header_id|>

What is in front of the Notre Dame Main Building?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I am unable to verify what is in front of the Notre Dame Main Building.


In [6]:
def extract_assistant_response(output_text):
    # Extract content after the assistant header
    if "<|start_header_id|>assistant<|end_header_id|>" in output_text:
        return output_text.split("<|start_header_id|>assistant<|end_header_id|>")[1].strip()
    return output_text

extract_assistant_response(outputs[0]["generated_text"])

'I am unable to verify what is in front of the Notre Dame Main Building.'

In [2]:
from mlflow.models import infer_signature
import mlflow

# last_run_id = mlflow.last_active_run().info.run_id
# Save a tokenizer without padding because it is only needed for training
tokenizer_no_pad = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-3.2-3B-Instruct", add_bos_token=True
)
with mlflow.start_run():
    # mlflow.log_params(self.lora_config.to_dict())
    mlflow.transformers.log_model(
        transformers_model=dict(
            model=base_model, tokenizer=tokenizer_no_pad
        ),
        artifact_path="model",
        signature=infer_signature(
            model_input={
                "role": "user",
                "content": "What is in front of the Notre Dame Main Building?",
            },
        ),
    )

Device set to use mps
2025/05/07 16:24:32 WARNING mlflow.utils.environment: Failed to resolve installed pip version. ``pip`` will be added to conda.yaml environment spec without a version specifier.


Error: [('/var/folders/r8/jzt55db94034zypvblzs5zk80000gn/T/tmp0w_o8yr0/model/model/model-00025-of-00029.safetensors', '/Users/lloydhamilton/Documents/PersonalProjects.nosync/tomorro_llm_tech/src/mlruns/0/7610a39315d84ce19b4a5bae981d1b22/artifacts/model/model/model-00025-of-00029.safetensors', "[Errno 28] No space left on device: '/var/folders/r8/jzt55db94034zypvblzs5zk80000gn/T/tmp0w_o8yr0/model/model/model-00025-of-00029.safetensors' -> '/Users/lloydhamilton/Documents/PersonalProjects.nosync/tomorro_llm_tech/src/mlruns/0/7610a39315d84ce19b4a5bae981d1b22/artifacts/model/model/model-00025-of-00029.safetensors'"), ('/var/folders/r8/jzt55db94034zypvblzs5zk80000gn/T/tmp0w_o8yr0/model/model/model-00020-of-00029.safetensors', '/Users/lloydhamilton/Documents/PersonalProjects.nosync/tomorro_llm_tech/src/mlruns/0/7610a39315d84ce19b4a5bae981d1b22/artifacts/model/model/model-00020-of-00029.safetensors', "[Errno 28] No space left on device: '/var/folders/r8/jzt55db94034zypvblzs5zk80000gn/T/tmp0w_o